# Estimación de Precios de Coches de Segunda Mano

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- La calidad de la predicción.
- La velocidad de la predicción.
- El tiempo requerido para el entrenamiento.

## Inicialización

In [47]:
# Carga todas las librerías

# Manipulación de datos
import math
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Modelado
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
import xgboost as xgb

## Cargar los datos

In [48]:
# Carga los datos en DataFrames
try:
    df = pd.read_csv('/datasets/car_data.csv')
except Exception:
    df = pd.read_csv('datasets/car_data.csv')

## Preparación de datos

In [49]:
# Imprime la información general/resumen sobre el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [50]:
# Imprime una muestra aleatoria de 5 filas del DataFrame
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
225743,01/04/2016 08:54,499,NaN,2016,NaN,55,polo,150000,5,petrol,volkswagen,no,01/04/2016 00:00,0,96106,07/04/2016 06:17
190504,10/03/2016 09:49,180,NaN,2016,manual,86,punto,150000,8,NaN,fiat,yes,10/03/2016 00:00,0,59558,10/03/2016 10:39
267381,20/03/2016 09:46,1,small,1998,auto,100,astra,150000,0,petrol,opel,NaN,20/03/2016 00:00,0,48529,21/03/2016 11:44
115020,28/03/2016 18:38,10000,sedan,2000,auto,286,7er,150000,1,lpg,bmw,no,28/03/2016 00:00,0,50667,02/04/2016 23:17
307704,17/03/2016 20:43,10600,convertible,2004,manual,163,slk,150000,7,petrol,mercedes_benz,no,17/03/2016 00:00,0,26129,01/04/2016 12:45


### Selección de columnas

In [51]:
# Seleccionar las columnas que se van a utilizar
chosen_columns = [
    "Price",
    "VehicleType", # Categórica
    "RegistrationYear",
    "Gearbox", # Categórica
    "Power",
    "Model",
    "Mileage",
    "FuelType", # Categórica
    "Brand",
    "NotRepaired"
]

# Crea un nuevo DataFrame con las columnas seleccionadas
df = df[chosen_columns]

# Cambiar los valores de la columna "NotRepaired" a booleanos
df["NotRepaired"] = df["NotRepaired"].replace({"yes": True, "no": False})

In [52]:
# Imprime la información general/resumen sobre el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Mileage           354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


### Imputación de datos

In [53]:
# Deshacerse de los registros con valores faltantes en la columna "Model"
df = df.dropna(subset=["Model"])

# Imputar los valores faltantes en la columna "VehicleType" usando la columna "Model"
df["VehicleType"] = df.groupby("Model")["VehicleType"].transform(lambda x: x.fillna(x.mode()[0]))

# Imputar los valores faltantes en la columna "Gearbox" usando la columna "Model"
df["Gearbox"] = df.groupby("Model")["Gearbox"].transform(lambda x: x.fillna(x.mode()[0]))

# Imputar los valores faltantes en la columna "FuelType" usando la columna "Model"
df["FuelType"] = df.groupby("Model")["FuelType"].transform(lambda x: x.fillna(x.mode()[0]))

# Imputar los valores faltantes en la columna "NotRepaired" con "no" considerando que son autos de segunda mano.
df["NotRepaired"] = df["NotRepaired"].fillna(False)

# Corregir el tipo de dato de la columna "NotRepaired"
df["NotRepaired"] = df["NotRepaired"].astype("uint8")

In [54]:
# Imprime la información general/resumen sobre el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334664 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             334664 non-null  int64 
 1   VehicleType       334664 non-null  object
 2   RegistrationYear  334664 non-null  int64 
 3   Gearbox           334664 non-null  object
 4   Power             334664 non-null  int64 
 5   Model             334664 non-null  object
 6   Mileage           334664 non-null  int64 
 7   FuelType          334664 non-null  object
 8   Brand             334664 non-null  object
 9   NotRepaired       334664 non-null  uint8 
dtypes: int64(4), object(5), uint8(1)
memory usage: 25.9+ MB


### Conclusión intermedia

La columna `Model` es de gran importancia para la imputación de los datos debido a que el modelo de un auto puede ser un buen indicador de las diferentes sus características, es por ello que se decidió utilizar dicha columna como base para realizar la imputación de algunas columnas con valores ausentes, además de que se eliminaron los registros que no contaban con este dato.

Los valores ausentes en la columna `NotRepaired` se rellenaron con "no" ya que dicho valor es la moda, además de que se está considerando la alta probabilidad de que un auto de segunda mano tenga reparaciones.

**NOTA:** Cabe mencionar que la columna `Power` contiene una cantidad considerable de valores en cero.

### Variables categóricas (Utilizar One Hot Encoding)

In [55]:
# Mostrar los diferentes valores de la columna "VehicleType"
df["VehicleType"].value_counts()

sedan          102434
small           85249
wagon           66365
bus             30901
convertible     19574
coupe           15631
suv             11703
other            2807
Name: VehicleType, dtype: int64

In [56]:
# Mostar los diferentes valores de la columna "Gearbox"
df["Gearbox"].value_counts()

manual    269911
auto       64753
Name: Gearbox, dtype: int64

In [57]:
# Mostar los diferentes valores de la columna "FuelType"
df["FuelType"].value_counts()

petrol      228196
gasoline    100555
lpg           4962
cng            544
hybrid         208
other          133
electric        66
Name: FuelType, dtype: int64

In [58]:
# Mostar los diferentes valores de la columna "Model"
df["Model"].value_counts()

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64

In [59]:
# Mostar los diferentes valores de la columna "Brand"
df["Brand"].value_counts()

volkswagen       73508
opel             38060
bmw              34979
mercedes_benz    30836
audi             28173
ford             24166
renault          17192
peugeot          10107
fiat              9141
seat              6672
skoda             5359
mazda             5263
smart             4978
citroen           4899
nissan            4734
toyota            4449
hyundai           3359
mini              3140
volvo             3101
mitsubishi        2862
honda             2662
kia               2352
suzuki            2186
alfa_romeo        2177
chevrolet         1644
chrysler          1350
dacia              878
daihatsu           767
subaru             718
porsche            716
jeep               651
land_rover         532
saab               510
daewoo             496
trabant            494
jaguar             491
lancia             448
rover              404
lada               210
Name: Brand, dtype: int64

In [60]:
# Seleccionar las columnas categóricas que se van a codificar con One-Hot Encoding
categorical_columns = [
    "VehicleType",
    "Gearbox",
    "FuelType",
    "Model", # 
    "Brand" #
]

# Codificar las columnas categóricas con One-Hot Encoding
df_ohe = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Imprime la información general/resumen sobre el DataFrame
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334664 entries, 0 to 354368
Columns: 306 entries, Price to Brand_volvo
dtypes: int64(4), uint8(302)
memory usage: 109.2 MB


## Conjuntos de entrenamiento y de prueba

In [61]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_ohe, test_ohe = train_test_split(df_ohe, test_size=0.25, random_state=12345)

# Dividir los conjuntos de entrenamiento y prueba en características y objetivos
train_features_ohe = train_ohe.drop("Price", axis=1)
train_targets_ohe = train_ohe["Price"]
test_features_ohe = test_ohe.drop("Price", axis=1)
test_targets_ohe = test_ohe["Price"]

## Entrenamiento del modelo 

### Encontrar los mejores hiperparámetros

In [62]:
# Crear una función para encontrar los mejores hiperparámetros
def find_best_params(model, param_grid, train_features, train_targets):
    # Realizar una búsqueda en cuadrícula de los hiperparámetros
    grid = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid.fit(train_features, train_targets)
    
    # Imprimir los mejores hiperparámetros y el mejor score
    print('Tipo de modelo:', type(model).__name__)
    print('Mejores parámetros:', grid.best_params_)
    print('Mejor score:', np.abs(grid.best_score_))
    print()

In [63]:
# Definir los hiperparámetros a probar
param_grid_lr = {}
param_grid_rf = {'n_estimators': np.arange(50, 201, 50), 'max_depth': np.arange(3, 15)}
param_grid_CatBoost = {'iterations': np.arange(50, 201, 50), 'depth': np.arange(3, 15)}
param_grid_XGBoost = {'n_estimators': np.arange(50, 201, 50), 'max_depth': np.arange(3, 15)}

In [64]:
# Encontrar los mejores hiperparámetros
# model_lr = LinearRegression()
# find_best_params(model_lr, param_grid_lr, train_features_ohe, train_targets_ohe)

# model_rf = RandomForestRegressor(random_state=12345)
# find_best_params(model_rf, param_grid_rf, train_features_ohe, train_targets_ohe)

# model_CatBoost = CatBoostRegressor(random_state=12345)
# find_best_params(model_CatBoost, param_grid_CatBoost, train_features_ohe, train_targets_ohe)

# model_XGBoost = xgb.XGBRegressor(random_state=12345)
# find_best_params(model_XGBoost, param_grid_XGBoost, train_features_ohe, train_targets_ohe)

```
Tipo de modelo: LinearRegression
Mejores parámetros: {}
Mejor score: 2279.3412678809937
```

```
Tipo de modelo: RandomForestRegressor
Mejores parámetros: {'max_depth': 14, 'n_estimators': 200}
Mejor score: 1153.716453910281
```

```
Tipo de modelo: CatBoostRegressor
Mejores parámetros: {'depth': 14, 'iterations': 200}
Mejor score: 1023.5344161101344
```

```
Tipo de modelo: XGBRegressor
Mejores parámetros: {'max_depth': 13, 'n_estimators': 200}
Mejor score: 1010.2158119709638
```

## Análisis del modelo

### Evaluación de RECM y velocidad

In [65]:
# Crear función para evaluar las predicciones del modelo, la velocidad de entrenamiento y predicción
def eval_model(model, train_features, train_target, test_features, test_target):
    start = time.time()
    model.fit(train_features, train_target)
    end = time.time()
    train_time = end - start
    start = time.time()
    predictions = model.predict(test_features)
    end = time.time()
    test_time = end - start
    rmse = math.sqrt(mean_squared_error(test_target, predictions))
    r2 = r2_score(test_target, predictions)
    print("RMSE: {:.2f}".format(rmse))
    print("R2: {:.2f}".format(r2))
    print("Tiempo de entrenamiento: {:.2f}".format(train_time))
    print("Tiempo de predicción: {:.2f}".format(test_time))

### Regresión lineal

In [21]:
# Crear modelo de regresión lineal
model_lr = LinearRegression()

# Entrenar y evaluar el modelo de regresión lineal
eval_model(model_lr, train_features_ohe, train_targets_ohe, test_features_ohe, test_targets_ohe)

RMSE: 3224.16
R2: 0.50
Tiempo de entrenamiento: 5.13
Tiempo de predicción: 0.11


### Bosque Aleatorio

In [22]:
# Crear modelo de regresión random forest
model_rf = RandomForestRegressor(random_state=12345, n_estimators=200, max_depth=14)

# Entrenar y evaluar el modelo de regresión random forest
eval_model(model_rf, train_features_ohe, train_targets_ohe, test_features_ohe, test_targets_ohe)

RMSE: 1838.29
R2: 0.84
Tiempo de entrenamiento: 327.35
Tiempo de predicción: 1.40


### CatBoost

In [25]:
# Crear modelo de regresión CatBoost
model_CatBoost = CatBoostRegressor(random_state=12345, iterations=200, depth=14, verbose=0)

# Entrenar y evaluar el modelo de regresión CatBoost
eval_model(model_CatBoost, train_features_ohe, train_targets_ohe, test_features_ohe, test_targets_ohe)

RMSE: 1687.89
R2: 0.86
Tiempo de entrenamiento: 14.48
Tiempo de predicción: 0.06


### XGBoost

In [46]:
# Crear modelo de regresión XGBoost
model_XGBoost = xgb.XGBRegressor(random_state=12345, n_estimators=200, max_depth=13, verbosity=0)

# Entrenar y evaluar el modelo de regresión XGBoost
eval_model(model_XGBoost, train_features_ohe, train_targets_ohe, test_features_ohe, test_targets_ohe)

RMSE: 1699.17
R2: 0.86
Tiempo de entrenamiento: 319.75
Tiempo de predicción: 0.29


### Conclusión intermedia

`CatBoost` parece ser el modelo más eficiente, proporcionando la menor puntuación de RMSE, el mayor valor de R2 y los tiempos de entrenamiento y predicción más bajos en comparación con los otros modelos.

# Conclusión General

Durante este proyecto para el servicio de venta de coches de segunda mano en `Rusty Bargain`:

1. Hemos limpiado e imputado datos, enfocándonos en la variable `Model` para mejorar la calidad de los datos.

2. Implementamos y comparamos varios modelos de regresión (regresión lineal, bosque aleatorio, CatBoost y XGBoost) para predecir el valor de mercado de los vehículos usados. Los modelos de potenciación del gradiente (CatBoost y XGBoost) mostraron un rendimiento superior.

3. Mejoramos la precisión de los modelos mediante ajuste de hiperparámetros, logrando un balance entre precisión y eficiencia.

4. Elegimos el modelo `CatBoost` basándonos en los resultados de las métricas de `RMSE`, `R2`, además del `tiempo de entrenamiento` y el `tiempo de predicción`.

En resumen, desarrollamos un modelo puede predecir con precisión el valor de mercado de los vehículos usados, proporcionando una valiosa herramienta para Rusty Bargain y destacando la importancia de un manejo cuidadoso de los datos y una consideración balanceada entre la precisión y la eficiencia del modelo.
